<a href="https://colab.research.google.com/github/CiriloRibeiro/QABlogPosts/blob/main/QAModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

In [11]:
import requests
from bs4 import BeautifulSoup
import time
import json

def getBlogPosts():
    url = 'https://improvado.io/blog'
    response = requests.get(url)
    print(response)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = []

    for link in soup.find_all('a', class_='link-block-12 w-inline-block'):
        links.append(link.get('href'))

    # use list comprehension to split each string at '/blog'
    links = [url + string.split('/blog')[1] for string in links]

    blog_posts = []

    for link in links:
        post_response = requests.get(link)
        post_soup = BeautifulSoup(post_response.content, 'html.parser')
        title = post_soup.find('h1', class_='c-h1 is-bold is-juicy-rosy is-blog-h1').text.strip()
        #author = post_soup.find('a', class_='blog-short-author-name author-name-blog').text.strip()
        #date = post_soup.find('div', class_='blog-post-date').text.strip()
        content = post_soup.find('div', class_='blog-body-block').get_text().strip()
        blog_posts.append({'title': title, 'content': content})
        #blog_posts.append({'title': title, 'author': author, 'date': date, 'content': content})
        time.sleep(0.1) # to be polite to the server and not make too many requests too quickly
    
    # Save the blog posts to a JSON file
    with open('blog_posts.json', 'w') as f:
        json.dump(blog_posts, f)

    return blog_posts

blog_posts = getBlogPosts()

<Response [200]>


In [24]:
# Load the blog posts JSON file
with open('blog_posts.json', 'r') as f:
    blog_posts = json.load(f)

In [25]:
blog_posts = blog_posts[0:21] #slices the blog posts
for post in blog_posts[0:5]:
   print(post['title'])

Salesforce Dashboard Development: Diving into Metrics, Tools, and Dashboard Types
Pinterest Ads Dashboard Overview & Metrics to Include
TikTok Ads Dashboard: Best Practices and KPIs
How to Get into Twitter Ads Dashboards & Reporting: Everything You Need To Know
How To Build A Comprehensive Amazon Ads Dashboard & What Metrics to Include


In [36]:
# a) Get predictions - Example from https://huggingface.co/deepset/roberta-base-squad2
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)
print(res)

{'score': 0.2524650990962982, 'start': 59, 'end': 132, 'answer': 'gives freedom to the user and let people easily switch between frameworks'}


In [34]:
# Train a question-answering model using the scraped blog posts
model_name = "deepset/tinyroberta-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
nlp = pipeline('question-answering', model=model, tokenizer=tokenizer)

i = 1
for post in blog_posts:
    input_text = post['title'] + '\n' + post['content']
    inputs = tokenizer.encode_plus(input_text, return_tensors='pt',max_length=512, truncation=True)
    question, text = post['title'], post['content']
    answer = nlp(question=question, context=text)
    score = nlp(question=question, context=text)
    print(f"Question {i}: {question}")
    print(f"Answer: {answer['answer']}")
    i += 1


Question 1: Salesforce Dashboard Development: Diving into Metrics, Tools, and Dashboard Types
Answer: Enhanced decision-making
Question 2: Pinterest Ads Dashboard Overview & Metrics to Include
Answer: everything you need to see about your paid advertising efforts
Question 3: TikTok Ads Dashboard: Best Practices and KPIs
Answer: A single source of truth for performance data
Question 4: How to Get into Twitter Ads Dashboards & Reporting: Everything You Need To Know
Answer: tool that allows advertisers to create, manage, and track their Twitter ad campaigns
Question 5: How To Build A Comprehensive Amazon Ads Dashboard & What Metrics to Include
Answer: selecting the right metrics
Question 6: How to Build an Influencer Marketing Dashboard (+ 8 metrics to track)
Answer: Marketing teams can track all essential metrics and ROI of campaigns
Question 7: The Top 7 Marketing Reporting Software Solutions for 2023
Answer: WhatagraphWhatagraph
Question 8: Instagram Analytics Dashboard: The Best Solut